In [ ]:
from __future__ import print_function
import os
import time
import logging
import argparse
import numpy as np
from visdom import Visdom
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *
from metric.loss import FitNet, AttentionTransfer, RKdAngle, RkdDistance

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
import models

# Student models:
# myNet, LeNet, FitNet

start_time = time.time()
# os.makedirs('./checkpoint', exist_ok=True)

# Training settings
parser = argparse.ArgumentParser(description='LR_adaptive_learning')

parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100'
                            ],
                    default='CIFAR10')
parser.add_argument('--teachers',
                    choices=['ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110',
                             'DenseNet121'
                            ],
                    default=['ResNet32', 'ResNet56', 'ResNet110'],
                    nargs='+')
parser.add_argument('--student',
                    choices=['ResNet20',
                             'myNet'
                            ],
                    default='ResNet20')

parser.add_argument('--kd_ratio', default=0.7, type=float)
parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--T', type=float, default=20.0, metavar='Temputure', help='Temputure for distillation')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:0', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=10, metavar='N', help='how many batches to wait before logging training status')

config = ['--epochs', '200', '--T', '5.0', '--device', 'cuda:0']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
load_dir = './checkpoint/' + args.dataset + '/'

# teachers model
teacher_models = []
for te in args.teachers:
    te_model = getattr(models, te)(num_classes=10)
#     print(te_model)
    te_model.load_state_dict(torch.load(load_dir + te_model.model_name + '.pth'))
    te_model.to(device)
    teacher_models.append(te_model)

st_model = getattr(models, args.student)()  # args.student()
st_model.to(device)

# logging
logfile = load_dir + 'ada_learning_' + st_model.model_name + '.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.array([0]),
    Y=np.array([0]),
    opts=dict(
            title='ada. loss',
        xtickmin=0,
#         xtickmax=1,
#         xtickstep=5,
        ytickmin=0,
#         ytickmax=1,
        ytickstep=0.5,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
    ),
    name="loss"
)

acc_win = vis.line(
    X=np.column_stack((0, 0)),
    Y=np.column_stack((0, 0)),
    opts=dict(
        title='ada. ACC',
        xtickmin=0,
#         xtickstep=5,
        ytickmin=0,
        ytickmax=100,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
        legend=['train_acc', 'test_acc']
    ),
    name="acc"
)


# adapter model
class Adapter():
    def __init__(self, in_models, pool_size):
        # representations of teachers
        pool_ch = pool_size[1]  # 64
        pool_w = pool_size[2]   # 8
        LR_list = []
        torch.manual_seed(1)
        self.theta = torch.randn(len(in_models), pool_ch).to(device)  # [3, 64]
        self.theta.requires_grad_(True)
   
        self.max_feat = nn.MaxPool2d(kernel_size=(pool_w, pool_w), stride=pool_w).to(device)
        self.W = torch.randn(pool_ch, 1).to(device)
        self.W.requires_grad_(True)
        self.val = False

    def loss(self, y, labels, weighted_logits, T=10.0, alpha=0.7):
        y = F.log_softmax(y/T)
        kd_loss = nn.KLDivLoss()(y, weighted_logits) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)
        angle_loss = angle_criterion(y, weighted_logits)
        dist_loss = dist_criterion(y, weighted_logits)
        ada_loss = kd_loss + angle_loss + dist_loss
        # Regularation
        if not self.val:
            ada_loss += 0.1 * (torch.sum(self.W * self.W) + torch.sum(torch.sum(self.theta * self.theta, dim=1), dim=0))
        return ada_loss
        
    def gradient(self, lr=0.01):
        self.W.data = self.W.data - lr * self.W.grad.data
        # Manually zero the gradients after updating weights
        self.W.grad.data.zero_()
        
    def eval(self):
        self.val = True
        self.theta.detach()
        self.W.detach()
    
    # input size: [64, 8, 8], [128, 3, 10]
    def forward(self, conv_map, te_logits_list):
        beta = self.max_feat(conv_map)
        beta = torch.squeeze(beta)  # [128, 64]
        
        latent_factor = []
        for t in self.theta:
            latent_factor.append(beta * t)
#         latent_factor = torch.stack(latent_factor, dim=0)  # [3, 128, 64]
        alpha = []
        for lf in latent_factor:  # lf.size:[128, 64]
            alpha.append(lf.mm(self.W))
        alpha = torch.stack(alpha, dim=0)  # [3, 128, 1]
        alpha = torch.squeeze(alpha).transpose(0, 1) # [128, 3]
        miu = F.softmax(alpha)  # [128, 3]
#         miu = torch.unsqueeze(miu, dim=2)
#         weighted_logits = miu * te_logits_list  # [128, 3, 10]
#         weighted_logits = torch.sum(weighted_logits, dim=1)
#         print(weighted_logits)
        
        return miu

# adapter instance
_,_,_,pool_m,_ = st_model(torch.randn(1, 3, 128, 128).to(device))  # get pool_size of student
# reate adapter instance
adapter = Adapter(teacher_models, pool_m.size())


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
train_set = getattr(datasets, args.dataset)(root='../data', train=True, download=True, transform=train_transform)
test_set = getattr(datasets, args.dataset)(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)
# optim
optimizer_W = optim.SGD([adapter.W], lr=args.lr, momentum=0.9)
optimizer_theta = optim.SGD([adapter.theta], lr=args.lr, momentum=0.9)
optimizer_sgd = optim.SGD(st_model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_sgd, gamma=0.1, milestones=[100, 150])
lr_scheduler2 = optim.lr_scheduler.MultiStepLR(optimizer_W, milestones=[40, 70])
lr_scheduler3 = optim.lr_scheduler.MultiStepLR(optimizer_theta, milestones=[40, 70])

# attention transfer loss
dist_criterion = RkdDistance().to(device)
angle_criterion = RKdAngle().to(device)
fitnet_criterion = [FitNet(32, 32), FitNet(32, 32),FitNet(32, 32)]
[f.to(device) for f in fitnet_criterion]


def train_adapter(n_epochs=70, model=st_model):
    print('Training adapter:')
    start_time = time.time()
    model.train()
    for epoch in range(n_epochs):
#         lr_scheduler.step(epoch)
        lr_scheduler2.step()
        lr_scheduler3.step()
        for i, (input, target) in enumerate(train_loader):

            input, target = input.to(device), target.to(device)
            # compute outputs
            b1, b2, b3, pool, output = model(input) # out_feat: 16, 32, 64, 64, - 
            st_maps = [b1, b2, b3, pool]
#             print('b1:{}, b2:{}, b3{}, pool:{}'.format(b1.size(), b2.size(), b3.size(), pool.size()))
# b1:torch.Size([128, 16, 32, 32]), b2:torch.Size([128, 32, 16, 16]), b3torch.Size([128, 64, 8, 8]), pool:torch.Size([128, 64, 1, 1])

            te_scores_list = []
            hint_maps = []
            fit_loss = 0
            for j,te in enumerate(teacher_models):
                te.eval()
                with torch.no_grad():
                    t_b1, t_b2, t_b3, t_pool, t_output = te(input)
#                 print('t_b1:{}, t_b2:{}, t_b3:{}, t_pool:{}'.format(t_b1.size(), t_b2.size(), t_b3.size(), t_pool.size()))
# t_b1:torch.Size([128, 16, 32, 32]), t_b2:torch.Size([128, 32, 16, 16]), t_b3torch.Size([128, 64, 8, 8]), t_pool:torch.Size([128, 64, 1, 1])
                hint_maps.append(t_b2)
                t_output = F.softmax(t_output/args.T)
                te_scores_list.append(t_output)
            te_scores_Tensor = torch.stack(te_scores_list, dim=1)  # size: [128, 3, 10]
            
            optimizer_sgd.zero_grad()
            optimizer_W.zero_grad()
            optimizer_theta.zero_grad()
            
            weight = adapter.forward(pool, te_scores_Tensor)
            weight_t = torch.unsqueeze(weight, dim=2)
            weighted_logits = weight_t * te_scores_Tensor  # [128, 3, 10]
            weighted_logits = torch.sum(weighted_logits, dim=1)
            # compute gradient and do SGD step
            ada_loss = adapter.loss(output, target, weighted_logits, T=args.T, alpha=args.kd_ratio)
            
            weight_f = F.softmax(torch.mean(weight, dim=0))  # weight for fit_loss
            for j in range(len(teacher_models)):
                fit_loss += fitnet_criterion[j](st_maps[1], hint_maps[j]) #weight_f[j] * 
            
#             fit_loss = fitnet_criterion[0](b2, hint_maps[0][3]) + fitnet_criterion[1](b3, hint_maps[1][3]) + fitnet_criterion(pool, hint_maps[2][3])
            loss = ada_loss + fit_loss
            
            loss.backward(retain_graph=True)
            optimizer_sgd.step()
            optimizer_W.step()
            optimizer_theta.step()
            
#          vis.line(np.array([loss.item()]), np.array([ep]), loss_win, update="append")
        log_out('epoch[{}/{}]adapter Loss: {:.4f}'.format(epoch, n_epochs, loss.item()))
#         vis.line(np.array([loss.item()]), np.array([epoch]), loss_win, update="append")
    end_time = time.time()
    log_out("--- adapter training cost {:.3f} mins ---".format((end_time - start_time)/60))


# train with multi-teacher
def train(epoch, model):
    print('Training:')
    # switch to train mode
    model.train()
    adapter.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        
        # compute outputs
        b1, b2, b3, pool, output = model(input)
        st_maps = [b1, b2, b3, pool]
        
        te_scores_list = []
        hint_maps = []
        fit_loss = 0
        for j,te in enumerate(teacher_models):
            te.eval()
            with torch.no_grad():
                t_b1, t_b2, t_b3, t_pool, t_output = te(input)
            hint_maps.append(t_b2)
            t_output = F.softmax(t_output/args.T)
            te_scores_list.append(t_output)
        te_scores_Tensor = torch.stack(te_scores_list, dim=1)  # size: [128, 3, 10]
        
        optimizer_sgd.zero_grad()
        
        weight = adapter.forward(pool, te_scores_Tensor)
        weight_t = torch.unsqueeze(weight, dim=2)
        weighted_logits = weight_t * te_scores_Tensor  # [128, 3, 10]
        weighted_logits = torch.sum(weighted_logits, dim=1)
        # compute gradient and do SGD step
        ada_loss = adapter.loss(output, target, weighted_logits, T=args.T, alpha=args.kd_ratio)
        
        weight_f = F.softmax(torch.mean(weight, dim=0))  # weight for fit_loss
#         for j in range(len(teacher_models)):
#             fit_loss += weight_f[j] * fitnet_criterion[j](st_maps[1], hint_maps[j])
        loss = ada_loss # + fit_loss

        loss.backward(retain_graph=True)
        optimizer_sgd.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        train_acc = accuracy(output.data, target.data)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(train_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg, train_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_,output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()

# """
print('StudentNet:\n')
print(st_model)
st_model.apply(weights_init_normal)
train_adapter(n_epochs=80)
# st_model.apply(weights_init_normal)
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
    log_out('current lr {:.5e}'.format(optimizer_sgd.param_groups[0]['lr']))
    lr_scheduler.step(epoch)
    train_loss, train_acc = train(epoch, st_model)
    # visaulize loss
    vis.line(np.array([train_loss]), np.array([epoch]), loss_win, update="append")
    _, test_acc, top1 = test(st_model)
    vis.line(np.column_stack((train_acc, top1)), np.column_stack((epoch, epoch)), acc_win, update="append")
    if top1 > best_acc:
        best_acc = top1
            
# release GPU memory
torch.cuda.empty_cache()
log_out("BEST ACC: {:.3f}".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))
# """

/home/data/yaliu/jupyterbooks/multi-KD/models/teacher/resnet20.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
StudentNet:

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:254: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:185: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:151: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release

epoch[0/80]adapter Loss: 1.8923
epoch[1/80]adapter Loss: 1.2946
epoch[2/80]adapter Loss: 1.2766
epoch[3/80]adapter Loss: 0.9707
epoch[4/80]adapter Loss: 0.8982
epoch[5/80]adapter Loss: 1.0691
epoch[6/80]adapter Loss: 0.9799
epoch[7/80]adapter Loss: 0.8803
epoch[8/80]adapter Loss: 0.8061
epoch[9/80]adapter Loss: 0.9717
epoch[10/80]adapter Loss: 0.7828
epoch[11/80]adapter Loss: 0.9247
epoch[12/80]adapter Loss: 0.8041
epoch[13/80]adapter Loss: 0.7219
epoch[14/80]adapter Loss: 0.8872
epoch[15/80]adapter Loss: 0.7206
epoch[16/80]adapter Loss: 0.7422
epoch[17/80]adapter Loss: 0.7651
epoch[18/80]adapter Loss: 0.6501
epoch[19/80]adapter Loss: 0.6966
epoch[20/80]adapter Loss: 0.7639
epoch[21/80]adapter Loss: 0.7309
epoch[22/80]adapter Loss: 0.6546
epoch[23/80]adapter Loss: 0.8422
epoch[24/80]adapter Loss: 0.6552
epoch[25/80]adapter Loss: 0.8946
epoch[26/80]adapter Loss: 0.7836
epoch[27/80]adapter Loss: 0.7034
epoch[28/80]adapter Loss: 0.6236
epoch[29/80]adapter Loss: 0.7160
epoch[30/80]adapter 

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:319: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:332: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

[230/391]	Time 0.184 (0.182)	Data 0.020 (0.020)	Loss 0.3596 (0.3462)	Prec@1 94.531 (95.789)
[240/391]	Time 0.179 (0.182)	Data 0.019 (0.020)	Loss 0.3535 (0.3459)	Prec@1 96.094 (95.799)
[250/391]	Time 0.179 (0.181)	Data 0.020 (0.020)	Loss 0.3769 (0.3456)	Prec@1 94.531 (95.811)
[260/391]	Time 0.178 (0.181)	Data 0.020 (0.020)	Loss 0.3158 (0.3453)	Prec@1 96.094 (95.800)
[270/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.3329 (0.3453)	Prec@1 96.094 (95.805)
[280/391]	Time 0.181 (0.181)	Data 0.020 (0.020)	Loss 0.3371 (0.3451)	Prec@1 96.875 (95.796)
[290/391]	Time 0.183 (0.181)	Data 0.019 (0.020)	Loss 0.3894 (0.3457)	Prec@1 93.750 (95.774)
[300/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.3482 (0.3462)	Prec@1 95.312 (95.767)
[310/391]	Time 0.148 (0.181)	Data 0.020 (0.020)	Loss 0.3500 (0.3462)	Prec@1 96.094 (95.772)
[320/391]	Time 0.154 (0.180)	Data 0.019 (0.020)	Loss 0.3454 (0.3468)	Prec@1 96.875 (95.741)
[330/391]	Time 0.180 (0.180)	Data 0.020 (0.020)	Loss 0.3990 (0.3475)	Prec@1 91.4

[180/391]	Time 0.180 (0.181)	Data 0.019 (0.020)	Loss 0.3509 (0.3440)	Prec@1 94.531 (95.852)
[190/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.3024 (0.3435)	Prec@1 96.094 (95.869)
[200/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.2987 (0.3441)	Prec@1 98.438 (95.826)
[210/391]	Time 0.182 (0.181)	Data 0.021 (0.020)	Loss 0.3208 (0.3443)	Prec@1 96.875 (95.812)
[220/391]	Time 0.183 (0.181)	Data 0.020 (0.020)	Loss 0.3259 (0.3441)	Prec@1 96.094 (95.846)
[230/391]	Time 0.188 (0.181)	Data 0.019 (0.020)	Loss 0.3657 (0.3446)	Prec@1 94.531 (95.810)
[240/391]	Time 0.182 (0.181)	Data 0.019 (0.020)	Loss 0.3371 (0.3453)	Prec@1 96.875 (95.766)
[250/391]	Time 0.181 (0.181)	Data 0.019 (0.020)	Loss 0.3282 (0.3454)	Prec@1 95.312 (95.742)
[260/391]	Time 0.181 (0.181)	Data 0.020 (0.020)	Loss 0.3932 (0.3449)	Prec@1 92.188 (95.756)
[270/391]	Time 0.182 (0.181)	Data 0.020 (0.020)	Loss 0.2960 (0.3456)	Prec@1 97.656 (95.739)
[280/391]	Time 0.180 (0.181)	Data 0.019 (0.020)	Loss 0.3519 (0.3464)	Prec@1 94.5

[130/391]	Time 0.183 (0.182)	Data 0.019 (0.020)	Loss 0.3495 (0.3398)	Prec@1 92.969 (95.992)
[140/391]	Time 0.179 (0.182)	Data 0.019 (0.020)	Loss 0.3733 (0.3416)	Prec@1 95.312 (95.933)
[150/391]	Time 0.181 (0.182)	Data 0.020 (0.020)	Loss 0.3246 (0.3425)	Prec@1 94.531 (95.897)
[160/391]	Time 0.184 (0.182)	Data 0.021 (0.020)	Loss 0.3474 (0.3429)	Prec@1 96.875 (95.856)
[170/391]	Time 0.178 (0.182)	Data 0.019 (0.020)	Loss 0.3533 (0.3427)	Prec@1 95.312 (95.879)
[180/391]	Time 0.180 (0.182)	Data 0.019 (0.020)	Loss 0.3561 (0.3430)	Prec@1 96.094 (95.900)
[190/391]	Time 0.183 (0.182)	Data 0.019 (0.020)	Loss 0.3386 (0.3427)	Prec@1 96.875 (95.897)
[200/391]	Time 0.182 (0.182)	Data 0.020 (0.020)	Loss 0.3666 (0.3432)	Prec@1 95.312 (95.888)
[210/391]	Time 0.182 (0.182)	Data 0.024 (0.020)	Loss 0.3747 (0.3439)	Prec@1 96.875 (95.857)
[220/391]	Time 0.178 (0.182)	Data 0.019 (0.020)	Loss 0.3207 (0.3443)	Prec@1 97.656 (95.825)
[230/391]	Time 0.178 (0.182)	Data 0.019 (0.020)	Loss 0.3699 (0.3439)	Prec@1 95.3

[80/391]	Time 0.184 (0.182)	Data 0.019 (0.020)	Loss 0.3269 (0.3422)	Prec@1 96.875 (95.901)
[90/391]	Time 0.185 (0.182)	Data 0.019 (0.020)	Loss 0.3541 (0.3420)	Prec@1 94.531 (95.862)
[100/391]	Time 0.186 (0.182)	Data 0.019 (0.020)	Loss 0.4237 (0.3444)	Prec@1 93.750 (95.738)
[110/391]	Time 0.180 (0.182)	Data 0.020 (0.020)	Loss 0.3547 (0.3458)	Prec@1 96.094 (95.693)
[120/391]	Time 0.179 (0.182)	Data 0.019 (0.020)	Loss 0.3553 (0.3454)	Prec@1 96.875 (95.713)
[130/391]	Time 0.179 (0.181)	Data 0.020 (0.020)	Loss 0.3522 (0.3464)	Prec@1 95.312 (95.694)
[140/391]	Time 0.178 (0.181)	Data 0.019 (0.020)	Loss 0.3404 (0.3465)	Prec@1 97.656 (95.673)
[150/391]	Time 0.182 (0.181)	Data 0.019 (0.020)	Loss 0.3492 (0.3467)	Prec@1 95.312 (95.654)
[160/391]	Time 0.181 (0.181)	Data 0.020 (0.020)	Loss 0.3816 (0.3459)	Prec@1 95.312 (95.696)
[170/391]	Time 0.181 (0.181)	Data 0.020 (0.020)	Loss 0.3992 (0.3468)	Prec@1 95.312 (95.664)
[180/391]	Time 0.177 (0.181)	Data 0.020 (0.020)	Loss 0.3247 (0.3464)	Prec@1 97.656

[30/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.3543 (0.3521)	Prec@1 94.531 (95.464)
[40/391]	Time 0.177 (0.181)	Data 0.019 (0.020)	Loss 0.3079 (0.3522)	Prec@1 98.438 (95.598)
[50/391]	Time 0.184 (0.181)	Data 0.019 (0.020)	Loss 0.3207 (0.3504)	Prec@1 97.656 (95.695)
[60/391]	Time 0.186 (0.181)	Data 0.021 (0.020)	Loss 0.3302 (0.3489)	Prec@1 95.312 (95.748)
[70/391]	Time 0.185 (0.182)	Data 0.019 (0.020)	Loss 0.3535 (0.3490)	Prec@1 95.312 (95.676)
[80/391]	Time 0.178 (0.182)	Data 0.019 (0.020)	Loss 0.3661 (0.3492)	Prec@1 94.531 (95.611)
[90/391]	Time 0.179 (0.182)	Data 0.020 (0.020)	Loss 0.4112 (0.3479)	Prec@1 92.969 (95.639)
[100/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.2974 (0.3469)	Prec@1 96.875 (95.676)
[110/391]	Time 0.179 (0.181)	Data 0.019 (0.020)	Loss 0.3038 (0.3471)	Prec@1 97.656 (95.686)
[120/391]	Time 0.183 (0.181)	Data 0.019 (0.020)	Loss 0.3341 (0.3480)	Prec@1 95.312 (95.609)
[130/391]	Time 0.184 (0.181)	Data 0.019 (0.020)	Loss 0.3832 (0.3479)	Prec@1 94.531 (95.